# Scraper para POD 1

Este documento contiene el scraper para obtener los sueldos medios por país.

Hemos obtenido los datos de Numbeo:

[https://www.numbeo.com/cost-of-living/country_price_rankings?itemId=105](https://www.numbeo.com/cost-of-living/country_price_rankings?itemId=105)


In [2]:
from IPython import display

display.Image(filename="/data/notebook_files/Numbeo1.png")

FileNotFoundError: FileNotFoundError: [Errno 2] No such file or directory: '/data/notebook_files/Numbeo1.png'

### Archivos del scraper

A continuación mostramos el código fuente del scraper:

In [1]:
!cat /data/notebook_files/sueldos/sueldos/items.py

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

from scrapy import Item, Field

class ScrapyItemSueldos(Item):
    sueldos = Field()


In [4]:
!cat /data/notebook_files/sueldos/sueldos/spiders/SpiderSueldos.py

import scrapy
from ..items import *

class SpiderSueldos(scrapy.Spider):
    name = 'SpiderSueldos'
    allowed_domains = ['http://www.numbeo.com/']
    start_urls = ['https://www.numbeo.com/cost-of-living/country_price_rankings?itemId=105']

    def parse(self, response):
        item = ScrapyItemSueldos()
        item['sueldos'] = []
        for fila in response.xpath('//tr[@class="tr_standard"]'):
            c = dict()
            c['pais'] = fila.xpath('./td[2]/text()').extract_first()
            c['sueldo'] = fila.xpath('./td[4]/text()').extract_first()
            item['sueldos'].append(c)
        return item


Los datos están organizados en una tabla, en la que cada fila tiene clase `tr_standard`.

Dentro de cada fila, los datos que nos interesan están en segunda posición (nombre del país), y cuarta posición (sueldo).

### Resultado

Tras ejecutar el scraper obtenemos una lista con los países y los sueldos medios respectivos. 

In [1]:
!cat /data/notebook_files/sueldos/sueldos/spiders/sueldos.json

[
{"sueldos": [{"pais": "\nSwitzerland", "sueldo": "\n6,221.54\u00a0$"}, {"pais": "\nLuxembourg", "sueldo": "\n4,224.22\u00a0$"}, {"pais": "\nSingapore", "sueldo": "\n4,004.48\u00a0$"}, {"pais": "\nAustralia", "sueldo": "\n3,639.57\u00a0$"}, {"pais": "\nUnited States", "sueldo": "\n3,592.23\u00a0$"}, {"pais": "\nDenmark", "sueldo": "\n3,494.11\u00a0$"}, {"pais": "\nNorway", "sueldo": "\n3,364.91\u00a0$"}, {"pais": "\nIceland", "sueldo": "\n3,155.89\u00a0$"}, {"pais": "\nHong Kong", "sueldo": "\n3,133.97\u00a0$"}, {"pais": "\nQatar", "sueldo": "\n3,113.51\u00a0$"}, {"pais": "\nNetherlands", "sueldo": "\n2,920.58\u00a0$"}, {"pais": "\nGermany", "sueldo": "\n2,889.17\u00a0$"}, {"pais": "\nIreland", "sueldo": "\n2,870.03\u00a0$"}, {"pais": "\nSweden", "sueldo": "\n2,862.29\u00a0$"}, {"pais": "\nNew Zealand", "sueldo": "\n2,821.94\u00a0$"}, {"pais": "\nCanada", "sueldo": "\n2,818.27\u00a0$"}, {"pais": "\nUnited Arab Emirates", "sueldo": "\n2,811.49\u00a0$"}, {"pais": "\nIsrael", "sueldo": "


Debemos realizar algunas correcciones antes de importar el archivo a un DataFrame:

* Eliminar los caracteres innecesarios: las newlines y los símbolos de dólar
* Eliminar los corchetes
* Indicar a Pandas la estructura del documento con `json_normalize` (solución obtenida de [StackOverflow](https://stackoverflow.com/questions/40588852/pandas-read-nested-json/40589031#40589031))
  

In [53]:
import json
import pandas as pd

with open("/data/notebook_files/sueldos/sueldos/spiders/sueldos_corregido.json") as data_file:    
    data = json.load(data_file)  

df = pd.json_normalize(data, 'sueldos', ['pais', 'sueldo'], 
                    record_prefix='datos_', errors='ignore')

df_limpia = df[['datos_pais', 'datos_sueldo']]

print(df_limpia)

df_limpia.to_csv("/data/notebook_files/resultados.csv")

        datos_pais datos_sueldo
0      Switzerland     6,221.54
1       Luxembourg     4,224.22
2        Singapore     4,004.48
3        Australia     3,639.57
4    United States     3,592.23
..             ...          ...
105          Nepal       221.89
106       Pakistan       200.50
107       Ethiopia       199.51
108        Nigeria       182.70
109          Syria        73.12

[110 rows x 2 columns]


Por último, tendremos que convertir los sueldos a números cuando queramos importar la tabla.

In [52]:
prueba = pd.read_csv('resultados.csv', sep=',')

prueba['datos_sueldo'] = prueba['datos_sueldo'].str.replace(',', '').astype(float)

prueba

,Unnamed: 0,datos_pais,datos_sueldo
0,0,Switzerland,6221.54
1,1,Luxembourg,4224.22
2,2,Singapore,4004.48
3,3,Australia,3639.57
4,4,United States,3592.23
...,...,...,...
105,105,Nepal,221.89
106,106,Pakistan,200.50
107,107,Ethiopia,199.51
108,108,Nigeria,182.70
